In [1]:
# Exam 2020/06/27 - Exercise #2 - version #2

In [10]:
inputPathCars = "exam_ex2_data/Cars.txt"
inputPathFailures = "exam_ex2_data/CarsFailures.txt"
outputPathPart1 = "outPart1"
outputPathPart2 = "outPart2"

In [11]:
# *****************************************
# Exercise 2 - Part 1
# *****************************************

In [12]:
# Read the content of CarsFailures.txt
failuresRDD = sc.textFile(inputPathFailures).cache()

In [13]:
# Select only the failures related to years 2017 and 2018 and failureType=Engine
# 2015/01/05,08:45,Car15,Engine
def years1718EngineFilter(line):
    fields = line.split(",")
    
    date = fields[0]
    failureType = fields[3]
    
    if ( (date.startswith("2018") or date.startswith("2017")) and failureType=="Engine"):
        return True
    else:
        return False
    
selectedFailuresRDD = failuresRDD.filter(years1718EngineFilter)    

In [14]:
# Count for each car the number of failures in years 2017 and 2018

# Map each input line to a pair
# key = carID
# value = (2017=1/0, 2018=1/0)

def pairCarIDCounters(line):
    fields = line.split(",")
    date = fields[0]
    carId= fields[2]
    
    if (date.startswith("2017")):
        return (carId, (1, 0))
    else:
        return (carId, (0, 1))
    
carsCountersRDD = selectedFailuresRDD.map(pairCarIDCounters)

In [15]:
# Sum the ones
carsTotFailures1718RDD = carsCountersRDD.reduceByKey(lambda v1,v2: (v1[0]+v2[0], v1[1]+v2[1]))

In [16]:
# Select the cars with #failures 2018># failurs 2017
selectedCarsRDD = carsTotFailures1718RDD.filter(lambda pair: pair[1][1]>pair[1][0])

In [17]:
# Read the content of Cars.txt
carsRDD = sc.textFile(inputPathCars)

In [18]:
# Extract (carId,model)
# Car12,Panda,FCA,Paris
carModelRDD = carsRDD.map(lambda line: (line.split(",")[0], line.split(",")[1]))

In [19]:
# Join the content of selectedCarsRDD with carModelRDD
joinSelectedRDD = selectedCarsRDD.join(carModelRDD)

In [20]:
# Select (carID,model) and store it in output first output folder
# carId is already the key. The must only change the value part in order to select only the model of the car
selectedCarsModels = joinSelectedRDD.mapValues(lambda value: value[1])

In [100]:
# Store the result
selectedCarsModels.saveAsTextFile(outputPathPart1)

In [101]:
# *****************************************
# Exercise 2 - Part 2
# *****************************************

In [102]:
# This function is already provided 
from datetime import datetime, timedelta
def previousDate(mydate):
    currentDate=datetime.strptime(mydate,"%Y/%m/%d")
    prevDate=currentDate-timedelta(days=1)
    return prevDate.strftime("%Y/%m/%d")

In [103]:
# Analyze all the failures and emit a pair (CarId, Date) from each failure
def carIdDate(line):
    # 2015/01/05,08:45,Car15,Engine
    fields = line.split(",")
    
    date = fields[0]
    carId = fields[2]
    
    return (carId, date)

carsDatesRDD = failuresRDD.map(carIdDate)

In [104]:
# Remove duplicates. The same car can have more than one failure per date
carsDatesDistinctRDD = carsDatesRDD.distinct()

In [105]:
# Emit two pairs from each input pair
# - ((carId,  date), +1)   - First element of the window that starts in this date
# - ((carId,date-1), +1)   - Second element of the window that starts in the previous date
def flatMapFunc(pair):
    returnedPairs = []
    
    carId = pair[0]
    currDate = pair[1]
    prevDate= previousDate(currDate)
    
    returnedPairs.append( ((carId, currDate), +1) )
    returnedPairs.append( ((carId, prevDate), +1) )
    
    return returnedPairs


windowsElementsRDD = carsDatesDistinctRDD.flatMap(flatMapFunc)

In [106]:
# Count the number of elements per windows
windowsNumElementsRDD = windowsElementsRDD.reduceByKey(lambda v1, v2: v1+v2)

In [107]:
# Select the windows with 2 elements
selectedWindowsRDD = windowsNumElementsRDD.filter(lambda pair: pair[1]==2)

In [108]:
# Store the key part of the selected pairs (carId, first date of the sequence)
selectedWindowsRDD.keys().saveAsTextFile(outputPathPart2)